In [6]:
import requests
import pandas as pd
import numpy as np

In [2]:
buying_Price = int(input("Input buying price in INR for BTC: "))

In [3]:
Selling_Price = int(input("What should be your selling price for BTC: "))

In [5]:
Volume = input("Enter the Volume of the BTC-INR: ")
Volume = float(Volume)

In [6]:
final_buying_price = (buying_Price)*Volume

In [7]:
final_selling_price = (Selling_Price)*Volume

In [8]:
makers_fee = 0.002*(buying_Price)*(Volume)


In [9]:
takers_fee = 0.002*(Selling_Price)*(Volume)

In [10]:
Final_fee = makers_fee+takers_fee

In [11]:
Profit_inc = (Selling_Price - buying_Price)*Volume
Profit_per = (Profit_inc/buying_Price)*100


In [12]:
Profit_Final = Profit_inc - Final_fee

In [13]:
url = "https://api.wazirx.com/api/v2/trades?market=btcinr"

In [14]:
response = requests.get(url)

In [15]:
data = response.json()[1]

In [16]:
response = requests.get(url)
data = response.json()[1]

In [19]:
data

{'id': 12405158,
 'market': 'btcinr',
 'price': '987700.0',
 'volume': '0.0099',
 'funds': '9778.23',
 'created_at': '2020-10-26T16:03:47+05:30',
 'side': None}

In [20]:
with open("BTC_INR.csv","a+") as fp:
    fp.write(f"\n{data['market']},{buying_Price},{Selling_Price},{data['price']},{Profit_inc},{Profit_Final},{data['created_at']},{data['volume']},{data['funds']},{data['id']}")


In [ ]:
while True:
    response = requests.get(url)
    data = response.json()[1]
    df = pd.read_csv('BTC_INR.csv', index_col=0)
    cur_Profit_inc = (float(data['price']) - buying_Price)*Volume
    cur_Profit_per = (Profit_inc/buying_Price)*100
    cur_makers_fee = 0.002*(buying_Price)*(Volume)
    cur_takers_fee = 0.002*(float(data['price']))*(Volume)
    cur_Final_fee = cur_makers_fee + cur_takers_fee
    cur_Profit_Final = cur_Profit_inc - cur_Final_fee
    if (data["id"] !=  df.iloc[-1,-1]):
        with open("BTC_INR.csv","a+") as fp:
            fp.seek(0,0)
            fp.write(f"\n{data['market']},{buying_Price},{Selling_Price},{data['price']},{cur_Profit_inc},{cur_Profit_Final},{data['created_at']},{data['volume']},{data['funds']},{data['id']}")


SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials v13sm10472885pgl.6 - gsmtp')

In [ ]:
df = pd.read_csv('BTC_INR.csv', index_col=0)

In [ ]:
df

In [7]:
ticker_url = "https://api.wazirx.com/api/v2/tickers"

In [18]:
response_ticker = requests.get(ticker_url)

In [19]:
data_ticker = response_ticker.json()["btcinr"]

In [20]:
data_ticker

{'base_unit': 'btc',
 'quote_unit': 'inr',
 'low': '980751.0',
 'high': '999475.0',
 'last': '999333.0',
 'type': 'SPOT',
 'open': 985989.0,
 'volume': '64.5483',
 'sell': '999000.0',
 'buy': '997099.0',
 'at': 1603721106,
 'name': 'BTC/INR'}